In [1]:
# You can also see https://community.frax.finance/info/airdrop for an interactive JS version
from cache import cache
from collections import defaultdict
from decimal import Decimal
import httpx
import json
from pprint import pprint
import sys

In [2]:
# Let's get our json files
frax_data = json.loads(cache.get('https://raw.githubusercontent.com/FraxFinance/frax-solidity/master/fpi-airdrop-snapshot.json')['value'])
cvx_data = json.loads(cache.get('https://raw.githubusercontent.com/convex-eth/platform/main/airdrop/fpis/cvxfxs_14246086.json')['value'])

In [3]:
# Store some wallets
fpis_wallets = defaultdict(dict)
cvxfxs_wallets =  defaultdict(dict)

In [4]:
# Frax FPIS stats
total_fpis = Decimal(0.0)
for key in frax_data:
  key_amt = Decimal(frax_data[key]['weight'])/10**18
  total_fpis += key_amt
print(f'Total FPIS Weight: {total_fpis:15,.2f}')
print()

# Let's see who has >1% of the FIPS distribution...
whale_pct = Decimal(0.0)
whales = 0
for key in frax_data:
  key_amt = Decimal(frax_data[key]['weight'])/10**18
  key_pct = 100*key_amt/total_fpis
  fpis_wallets[key]['amount'] = key_amt
  fpis_wallets[key]['percent'] = key_pct
  fpis_wallets[key]['frax'] = 1
  if key_pct > 1:
    whale_pct += key_pct
    whales +=1

total_percent = Decimal(0.0)
for address in sorted(fpis_wallets, key=lambda a: fpis_wallets[a]['amount'], reverse=True):
  if fpis_wallets[address]["percent"] < 1:
    break
  total_percent += fpis_wallets[address]["percent"]
  print(f'{address} : {fpis_wallets[address]["amount"]:15,.2f} : {fpis_wallets[address]["percent"]:6.3f}% : {total_percent:6.3f}%')

print(f'The top {whales} accounts received {whale_pct:.2f}% of total FPIS airdrop')

Total FPIS Weight:  120,777,195.10

0x59CFCD384746ec3035299D90782Be065e466800B :   25,225,040.83 : 20.886% : 20.886%
0xe0dd07B181cA77eD9063821b4D2D28a85f722f5f :    7,403,142.96 :  6.130% : 27.015%
0xdc87397165B4f0F6d586f1bf60FA37494A362A6a :    6,324,937.95 :  5.237% : 32.252%
0x88E863d4572d2DaE27db81E98837a9dbeb0e7a12 :    5,833,618.98 :  4.830% : 37.082%
0xb744bEA7E6892c380B781151554C7eBCc764910b :    5,677,326.95 :  4.701% : 41.783%
0x9AA7Db8E488eE3ffCC9CdFD4f2EaECC8ABeDCB48 :    5,108,404.84 :  4.230% : 46.012%
0x9c5083dd4838E120Dbeac44C052179692Aa5dAC5 :    4,298,491.55 :  3.559% : 49.571%
0x10c16c7B8b1DDCFE65990ec822DE4379dd8a86dE :    3,904,666.00 :  3.233% : 52.804%
0xCC4e62029e0491a61a7ac1b60ab1721F6df841E3 :    2,818,672.38 :  2.334% : 55.138%
0x94712699C06Ec46b09f31B1aECd078049Dad3Fe3 :    2,349,313.51 :  1.945% : 57.083%
0xb55794c3bef4651b6cBc78b64a2Ef6c5c67837C3 :    2,019,932.38 :  1.672% : 58.756%
0x420A6C1c79a6Ce31ED9dC1C4343310C97b378F83 :    1,987,434.76 :  1.646% : 

In [6]:
# 0x59CFCD384746ec3035299D90782Be065e466800B is Convex so let's get better numbers
cvxfxs_fpis = fpis_wallets['0x59CFCD384746ec3035299D90782Be065e466800B']['amount']
# We should del this but this makes re-running easier
fpis_wallets['0x59CFCD384746ec3035299D90782Be065e466800B']['amount'] = Decimal(0.0)

# First lets get our cvx total
cvx_total = Decimal(0.0)
for address in cvx_data['addresses']:
  cvx_amt = Decimal(cvx_data['addresses'][address])/10**18
  cvx_total += cvx_amt
  
# now we have the cvxfxs_fpis which is the FPIS weight (25225040.832722331323958487)
# and we have the cvx_total for the cvxFXS weight (6334170.246284147692985248)
# and we can calculate the combined amount and % of FPIS weight
for address in cvx_data['addresses']:
  cvx_amt = Decimal(cvx_data['addresses'][address])/10**18
  key_amt = cvx_amt * (cvxfxs_fpis/cvx_total)
  key_pct = 100*key_amt/total_fpis
  try:
    fpis_wallets[address]['amount'] += key_amt
    fpis_wallets[address]['percent'] += key_pct
  except:
    fpis_wallets[address]['amount'] = key_amt
    fpis_wallets[address]['percent'] = key_pct

  fpis_wallets[address]['cvxfxs'] = 1

  cvxfxs_wallets[address]['amount'] = key_amt
  cvxfxs_wallets[address]['percent'] = key_pct
  
print()
print('FPIS Whales (1%+ of airdrop)')
whales = 0
total_percent = Decimal(0.0)
for address in sorted(fpis_wallets, key=lambda a: fpis_wallets[a]['amount'], reverse=True):
  if fpis_wallets[address]["percent"] < 1:
    break
  total_percent += fpis_wallets[address]["percent"]
  holdings = []
  if 'frax' in fpis_wallets[address]:
    holdings.append('FXS')
  if 'cvxfxs' in fpis_wallets[address]:
    holdings.append('cvxFXS')
  if holdings:
    holdings = ' '.join(holdings)
  else:
    holdings = ''
  whales +=1
  print(f'{address} : {fpis_wallets[address]["amount"]:13,.2f} : {fpis_wallets[address]["percent"]:6.3f}% : {total_percent:6.3f}% : {holdings}')
print(f'The top {whales} accounts received {total_percent:.2f}% of total FPIS airdrop')

cvx_holders = len(cvx_data['addresses'])
print()
print(f'Frax airdrop addresses   : {len(frax_data)}')
print(f'cvxFXS airdrop addresses : {cvx_holders}')
print(f'Total airdrop addresses  : {len(fpis_wallets)}')


FPIS Whales (1%+ of airdrop)
0xe0dd07B181cA77eD9063821b4D2D28a85f722f5f :  7,403,142.96 :  6.130% :  6.130% : FXS
0x88E863d4572d2DaE27db81E98837a9dbeb0e7a12 :  6,411,982.15 :  5.309% : 11.439% : FXS cvxFXS
0xdc87397165B4f0F6d586f1bf60FA37494A362A6a :  6,324,937.95 :  5.237% : 16.675% : FXS
0xb744bEA7E6892c380B781151554C7eBCc764910b :  5,677,326.95 :  4.701% : 21.376% : FXS
0x9AA7Db8E488eE3ffCC9CdFD4f2EaECC8ABeDCB48 :  5,108,404.84 :  4.230% : 25.606% : FXS
0x9c5083dd4838E120Dbeac44C052179692Aa5dAC5 :  4,298,491.55 :  3.559% : 29.165% : FXS
0x10c16c7B8b1DDCFE65990ec822DE4379dd8a86dE :  3,904,666.00 :  3.233% : 32.398% : FXS
0xCC4e62029e0491a61a7ac1b60ab1721F6df841E3 :  2,818,672.38 :  2.334% : 34.731% : FXS
0x5028D77B91a3754fb38B2FBB726AF02d1FE44Db6 :  2,620,672.94 :  2.170% : 36.901% : FXS cvxFXS
0x94712699C06Ec46b09f31B1aECd078049Dad3Fe3 :  2,349,313.51 :  1.945% : 38.846% : FXS
0xb55794c3bef4651b6cBc78b64a2Ef6c5c67837C3 :  2,019,932.38 :  1.672% : 40.519% : FXS
0x420A6C1c79a6Ce31ED9

In [13]:
# Just for fun, let's see the top weighting for cvxfxs holders...
print()
print('cvxFXS Top 20')
whales = 0
total_percent = Decimal(0.0)
for address in sorted(cvxfxs_wallets, key=lambda a: cvxfxs_wallets[a]['amount'], reverse=True):
  if whales >= 20:
    break
  total_percent += fpis_wallets[address]["percent"]
  holdings = []
  if 'cvxfxs' in fpis_wallets[address]:
    holdings.append('cvxFXS')
  if 'frax' in fpis_wallets[address]:
    holdings.append('FXS')
  if holdings:
    holdings = ' '.join(holdings)
  else:
    holdings = ''
  whales +=1
  print(f'{address} : {fpis_wallets[address]["amount"]:13,.2f} : {fpis_wallets[address]["percent"]:6.3f}% : {total_percent:6.3f}% : {holdings}')
print(f'The top {whales} accounts received {total_percent:.2f}% of total FPIS airdrop')


cvxFXS Top 20
0x5028D77B91a3754fb38B2FBB726AF02d1FE44Db6 :  2,620,672.94 :  2.170% :  2.170% : cvxFXS FXS
0xe2C8f362154aacE6144Cb9d96f45b9568e0Ea721 :  1,659,824.00 :  1.374% :  3.544% : cvxFXS
0x482ba6028f0332aAFEA3cFc040D797350975eFea :  1,515,246.79 :  1.255% :  4.799% : cvxFXS
0xa42e82C8Cb31068B240772eC69685ffc59b7Fd11 :  1,128,078.84 :  0.934% :  5.733% : cvxFXS FXS
0xAAc0aa431c237C2C0B5f041c8e59B3f1a43aC78F :    600,140.93 :  0.497% :  6.230% : cvxFXS
0x88E863d4572d2DaE27db81E98837a9dbeb0e7a12 :  6,411,982.15 :  5.309% : 11.539% : cvxFXS FXS
0x005fb56Fe0401a4017e6f046272dA922BBf8dF06 :    626,900.94 :  0.519% : 12.058% : cvxFXS FXS
0x3F3E305C4Ad49271EBDA489dd43d2c8F027d2D41 :    477,884.99 :  0.396% : 12.453% : cvxFXS
0xa5E0Ecb5D075e86C3bf5d1B1042e154FEe9a4798 :    438,066.39 :  0.363% : 12.816% : cvxFXS
0x4FeF64Cdb12F7DF11EDF18A3817690f7c9B8317e :    570,382.02 :  0.472% : 13.288% : cvxFXS FXS
0xA52Fd396891E7A74b641a2Cb1A6999Fcf56B077e :    387,322.03 :  0.321% : 13.609% : cvxF